In [1]:
import os
import zipfile
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from pycocotools.coco import COCO
import shutil

# --------------------------------------------
# 1. Setup Kaggle API credentials
# --------------------------------------------

# Make sure you have kaggle.json in the same folder as this script
# You can download it from your Kaggle account > Settings > API > Create New API Token
KAGGLE_JSON_PATH = "kaggle.json"  # path to your local kaggle.json

HOME = os.path.expanduser("~")
KAGGLE_DIR = os.path.join(HOME, ".kaggle")

# Create ~/.kaggle directory if not exists
os.makedirs(KAGGLE_DIR, exist_ok=True)

# Copy kaggle.json there
shutil.copy(KAGGLE_JSON_PATH, KAGGLE_DIR)
os.chmod(os.path.join(KAGGLE_DIR, "kaggle.json"), 0o600)

print("✅ Kaggle API key set up successfully!")

# --------------------------------------------
# 2. Install dependencies (optional)
# Run this only once or manually in terminal:
# pip install kaggle pycocotools matplotlib pillow tqdm
# --------------------------------------------


# --------------------------------------------
# 3. Dataset setup
# --------------------------------------------
DATASET = "awsaf49/coco-2017-dataset"
SAVE_DIR = "dataset"
os.makedirs(SAVE_DIR, exist_ok=True)

# Directories for extracted data
os.makedirs("train2017", exist_ok=True)
os.makedirs("val2017", exist_ok=True)
os.makedirs("annotations", exist_ok=True)
os.makedirs("train_subset", exist_ok=True)


# --------------------------------------------
# 4. Download and unzip COCO 2017 datasets
# --------------------------------------------
print("⬇️ Downloading COCO 2017 annotations...")
os.system("wget -q http://images.cocodataset.org/annotations/annotations_trainval2017.zip")
os.system("unzip -qq annotations_trainval2017.zip -d annotations")

print("⬇️ Downloading COCO 2017 validation set...")
os.system("wget -q http://images.cocodataset.org/zips/val2017.zip")
os.system("unzip -qq val2017.zip -d val2017")

print("⬇️ Downloading COCO 2017 training set subset (~20k images)...")
os.system("wget -q http://images.cocodataset.org/zips/train2017.zip")
os.makedirs("train_subset", exist_ok=True)
os.system('unzip -qq train2017.zip "train2017/00*.jpg" "train2017/01*.jpg" "train2017/02*.jpg" -d train_subset')

print("✅ Extracted ~20,000 COCO images + annotations successfully!")


✅ Kaggle API key set up successfully!
⬇️ Downloading COCO 2017 annotations...
⬇️ Downloading COCO 2017 validation set...
⬇️ Downloading COCO 2017 training set subset (~20k images)...
✅ Extracted ~20,000 COCO images + annotations successfully!


In [ ]:
import zipfile
import os

zip_path = "annotations_trainval2017.zip"
extract_dir = "/annotations"

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("✅ Annotations extracted successfully!")

FileNotFoundError: [Errno 2] No such file or directory: '/annotations_trainval2017.zip'

In [5]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from pycocotools.coco import COCO

# ------------------------------------------------
# 1. Set local paths
# ------------------------------------------------
# Adjust these if your folder structure is different
ANNOTATION_FILE = "annotations/annotations/instances_val2017.json"
IMAGE_DIR = "val2017/val2017"
NUM_SAMPLES = 50

# ------------------------------------------------
# 2. Load COCO annotations
# ------------------------------------------------
print("\n🔹 Loading COCO annotations...")
if not os.path.exists(ANNOTATION_FILE):
    raise FileNotFoundError(f"Annotation file not found at {ANNOTATION_FILE}")

coco = COCO(ANNOTATION_FILE)

# ------------------------------------------------
# 3. Visualization function
# ------------------------------------------------
def visualize_images(coco, img_dir, num_samples=50):
    img_ids = coco.getImgIds()
    random.shuffle(img_ids)
    shown = 0

    for img_id in img_ids:
        if shown >= num_samples:
            break

        img_info = coco.loadImgs(img_id)[0]
        img_path = os.path.join(img_dir, img_info["file_name"])
        if not os.path.exists(img_path):
            continue

        ann_ids = coco.getAnnIds(imgIds=img_info["id"])
        anns = coco.loadAnns(ann_ids)
        if not anns:
            continue

        # Load original image
        img = np.array(Image.open(img_path).convert("RGB"))

        # Build combined mask (object regions)
        mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
        for ann in anns:
            mask = np.maximum(mask, coco.annToMask(ann))

        # Create masked image (keep objects, blacken background)
        masked_img = img.copy()
        masked_img[mask == 0] = 0

        # Display side by side
        fig, ax = plt.subplots(1, 2, figsize=(8, 4))
        ax[0].imshow(img)
        ax[0].set_title("Original Image")
        ax[0].axis("off")

        ax[1].imshow(masked_img)
        ax[1].set_title("Masked (Object Highlighted)")
        ax[1].axis("off")

        plt.tight_layout()
        plt.show()

        shown += 1
        print(f"✅ Visualized {shown}/{num_samples}")

# ------------------------------------------------
# 4. Run visualization
# ------------------------------------------------
print("\n🔹 Visualizing 50 image–mask pairs...")
visualize_images(coco, IMAGE_DIR, NUM_SAMPLES)
print("\n🎯 Milestone 1 (Week 1) successfully completed!")



🔹 Loading COCO annotations...


FileNotFoundError: Annotation file not found at annotations/annotations/instances_val2017.json